<a href="https://colab.research.google.com/github/LaviWhite/Story-Annotation/blob/main/GRU_Product_Comparsion_Review/GRU_Product_Comparsion_Review_supervised_sample_weight_75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Sampling

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Dissertation/Original.xlsx')
df.head()

,review_cnt,is_hot,publish_time,id,is_original,pic_urls,profilers_id,author_url,site_id,fans_cnt,...,red_id,video_cnt,item_id,follow_cnt,coin_cnt,data_type,user_location,entity_keywords.keyword,entity_keywords.type,cooperation_brand_names
0,84.0,NaN,2021-02-10 12:10:00,60235cc900000000010282bf,普通笔记,[http://sns-img-qc.xhscdn.com/e12a7183-0d2b-3c...,[741974],http://www.xiaohongshu.com/user/profile/5a2ff3...,146510,464770,...,420599937,94,60235cc900000000010282bf,185,2386973,电商笔记,浙江 杭州,NaN,NaN,NaN
1,76.0,NaN,2021-02-06 12:05:00,601e158f000000000101f856,普通笔记,[http://sns-img-hw.xhscdn.com/3795c2fe-1819-34...,[739701],http://www.xiaohongshu.com/user/profile/5a2ff3...,146510,457941,...,420599937,92,601e158f000000000101f856,185,2341527,电商笔记,浙江 杭州,NaN,NaN,NaN
2,0.0,爆文,2021-02-16 14:33:00,602b6729000000002103e7ff,视频笔记,[http://sns-img-qc.xhscdn.com/6fd88377-9ffa-34...,[747652],http://www.xiaohongshu.com/user/profile/5e8f86...,146510,555822,...,T654837,125,602b6729000000002103e7ff,7,2106984,电商笔记,广西 南宁,NaN,NaN,NaN
3,1182.0,爆文,2021-02-10 17:00:00,6023a0c7000000002103ecd0,视频笔记,"[""http://sns-img-qc.xhscdn.com/9f6a4597-1fd3-3...","[""859906""]",http://www.xiaohongshu.com/user/profile/565f83...,146510,535219,...,494694428,438,6023a0c7000000002103ecd0,111,2150628,电商笔记,中国,NaN,NaN,NaN
4,523.0,爆文,2021-02-03 22:32:00,601ab3fd000000000101f4b2,视频笔记,"[""http://sns-img-qc.xhscdn.com/38e07503-6677-3...","[""859912""]",http://www.xiaohongshu.com/user/profile/55d6d2...,146510,856274,...,Qbaby414,416,601ab3fd000000000101f4b2,1,3099241,电商笔记,陕西 西安,NaN,NaN,NaN


In [ ]:
sample = df['content'].sample(n=10000, random_state=1)
sample.to_excel("/content/randomsample.xlsx")

# Mount Drive for saving

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [ ]:
!pip install jieba

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download zh_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 71.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/8f/bc/213638f0242858d936c171f4e32e431a6fbbe703184a944a81c065cdc414/spacy-3.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.
2023-07-22 07:34:40.299322: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variab

In [ ]:
import pandas as pd
import jieba
import jieba.analyse
from nltk.corpus import stopwords
import numpy as np

In [ ]:
def stopwordslist():
        stopwords = [line.strip() for line in open('/content/drive/MyDrive/Dissertation/stopwords.txt',encoding='utf-8').readlines()]
        return stopwords

def seg_depart(sentence):
    sentence_depart = jieba.lcut(sentence)
    stopwords = stopwordslist()
    outstr=''
    for word in sentence_depart:
        if word not in stopwords:
            outstr += word
            outstr += ' '
    return outstr

# Text Segmentation of Train set (without label)

In [ ]:
# Split into testing set and training without label set and output as txt file

from sklearn.model_selection import train_test_split
dt = pd.read_excel('/content/drive/MyDrive/Dissertation/Original.xlsx')
X = dt['content']

train, test = train_test_split(X, test_size=0.005, random_state=2)

#test.to_excel("/content/drive/MyDrive/Dissertation/test_verification.xlsx")

In [ ]:
output_train = []

for line in train.astype(str):
  line_seg=seg_depart(line)
  output_train.append(line_seg)
print('success！') # output_train is the segmented list for training(without label) set

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.694 seconds.
DEBUG:jieba:Loading model cost 0.694 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


success！


In [ ]:
file = open('/content/drive/MyDrive/Dissertation/Train_nolabel.txt','w')
for item in output_train:
	file.write(item+"\n")
file.close()
print('success')

success


# Text Segmentation of Testing set (without label)

In [ ]:
output_test = []

for line in test.astype(str):
  line_seg=seg_depart(line)
  output_test.append(line_seg)
print('success！') # output_test is the segmented list for testing set

NameError: ignored

In [ ]:
file = open('/content/drive/MyDrive/Dissertation/Test.txt','w')
for item in output_test:
	file.write(item+"\n")
file.close()
print('success！')

# Text Segmentation of Training Set (with label)

In [ ]:
dr = pd.read_excel('/content/drive/MyDrive/Dissertation/randomsample.xlsx', index_col='index')
#content = df['content'].tolist()

dr1 = dr.loc[dr['label'] == 1]
dr0 = dr.loc[dr['label'] == 0]

print('Sample size before downsize: '+str(len(dr)))
print('Positive sample size before downsize: '+str(len(dr1)))
print('Negative sample size before downsize: '+str(len(dr0)))

dr0 = dr0.sample(frac=0.032, replace=True, random_state=1) #downsize the major group
# 0.015 resulted 15 positive sample


df = pd.concat([dr1,dr0], ignore_index=True, sort=False)

df = df.sample(frac=1).reset_index() #shuffle the order of the trainning sample

label = df['label'].tolist()

print('Sample size after downsize: '+str(len(df)))
print('Positive sample size after downsize: '+str(len(df.loc[df['label']== 1])))
print('Negative sample size after downsize: '+str(len(df.loc[df['label']== 0])))
print('Proportion for Positive sample after downsize: '+str(len(df.loc[df['label']== 1])/len(df)))

Sample size before downsize: 10000
Positive sample size before downsize: 879
Negative sample size before downsize: 9121
Sample size after downsize: 1171
Positive sample size after downsize: 879
Negative sample size after downsize: 292
Proportion for Positive sample after downsize: 0.7506404782237404


In [ ]:
output = []

for line in df['content'].astype(str):
  line_seg=seg_depart(line)
  output.append(line_seg) # output is the segmented list for training(with label) set
print('success！')

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.746 seconds.
DEBUG:jieba:Loading model cost 0.746 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


success！


In [ ]:
ele = [" +++$+++ "]
element = ele * len(df) # add separator element to separate label and content
#print(element)

concat_function = lambda x,y: str(x) + "" + y # change label as string and concat separator element
concat_func = lambda x,y: x + "" + y # concat label and training set

a = list(map(concat_function,label,element))

labels = list(map(concat_func,a,output))

In [ ]:
# output as txt file Train with label

file = open('/content/drive/MyDrive/Dissertation/Train_label.txt','w')
for item in labels:
	file.write(item+"\n")
file.close()

print('success！')

success！


# Model Training

In [ ]:
import warnings
warnings.filterwarnings('ignore')
path_prefix = './'

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F

def load_training_data(path='/content/drive/MyDrive/Dissertation/Train_label.txt'):
    # Read training data
    if 'Train_label' in path:
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        return x, y
    else:
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='/content/drive/MyDrive/Dissertation/Test.txt'):
    # Read testing data
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")).strip() for line in lines]
        X = [sen.split(' ') for sen in X]
    return X

def evaluation(outputs, labels):
    #outputs => probability (float)
    #labels => labels
    outputs[outputs>=0.5] = 1 # Negtive Sentiment
    outputs[outputs<0.5] = 0 # Positive Sentiment
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

# Train Word to Vector

In [ ]:
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(x):
    model = word2vec.Word2Vec(x, vector_size=250, window=5, min_count=5, workers=12)
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data('/content/drive/MyDrive/Dissertation/Train_label.txt')
    #train_x_no_label = load_training_data('/content/drive/MyDrive/Dissertation/Train_nolabel.txt')

    print("loading testing data ...")
    test_x = load_testing_data('/content/drive/MyDrive/Dissertation/Test.txt')

    #model = train_word2vec(train_x + train_x_no_label + test_x)
    model = train_word2vec(train_x + test_x)

    print("saving model ...")
    model.save(os.path.join(path_prefix, '/w2v_all.model'))

loading training data ...
loading testing data ...
saving model ...


# Data Preprocess

In [ ]:
from torch import nn
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # load word to vector model
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # add word into embedding
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
    def make_embedding(self, load=True):
        print("Get embedding ...")
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError

        for i, word in enumerate(self.embedding.wv.key_to_index):
            print('get words #{}'.format(i+1), end='\r')
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding.wv[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    def pad_sequence(self, sentence):
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self):
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        # turn labels into tensors
        y = [int(label) for label in y]
        return torch.LongTensor(y)

# Dataset

In [ ]:
import torch
from torch.utils import data

class TwitterDataset(data.Dataset):
    """
    Expected data shape like:(data_num, data_len)
    Data can be a list of numpy array or a list of lists
    input data shape : (data_num, seq_len, feature_dim)

    __len__ will return the number of data
    """
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

# LSTM Model

In [ ]:
import torch
from torch import nn
class GRU_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(GRU_Net, self).__init__()
        # embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # Whether fix embedding
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                          nn.Linear(hidden_dim, 1),
                          nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.gru(inputs, None)
        # dimension of x (batch, seq_len, hidden_size)
        x = x[:, -1, :]
        x = self.classifier(x)
        return x

# Define Training

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    model.train() # set training mode
    criterion = nn.BCELoss() # Define loss function
    t_batch = len(train)
    v_batch = len(valid)
    optimizer = optim.Adam(model.parameters(), lr=lr) # set optimizer as Adam
    total_loss, total_acc, best_acc = 0, 0, 0
    epoch_num=range(1,6)#1,2,3,4,5
    train_loss=[]
    valid_loss=[]
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0

        # For training
        model.train() #set training mode
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long) # set device "cuda"
            labels = labels.to(device, dtype=torch.float) # set device "cuda"
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.squeeze()
            model.train()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            correct = evaluation(outputs, labels) # calculate accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
            	epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))
        train_loss.append(total_loss/t_batch)

        # For validation
        model.eval() # set validation mode
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) # set device "cuda"
                labels = labels.to(device, dtype=torch.float) # set device "cuda"
                outputs = model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels)
                correct = evaluation(outputs, labels)
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # if the result of validation is better than previous model, save the new model
                best_acc = total_acc
                torch.save(model, "{}/ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        valid_loss.append(total_loss/v_batch)

# Define Pesudo Labelling

In [ ]:
'''
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import math

def got_label(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            #outputs[outputs>=0.9] = 1 # cannot change into 1 first
            #outputs[outputs<0.1] = 0 # cannot change into 0 first

            ret_output += outputs.tolist() #cannot use int() as we need float

    return ret_output #pesudo_x, pesudo_y
'''

'\nimport torch\nfrom torch import nn\nimport torch.optim as optim\nimport torch.nn.functional as F\nimport math\n\ndef got_label(batch_size, test_loader, model, device):\n    model.eval()\n    ret_output = []\n    with torch.no_grad():\n        for i, inputs in enumerate(test_loader):\n            inputs = inputs.to(device, dtype=torch.long)\n            outputs = model(inputs)\n            outputs = outputs.squeeze()\n            #outputs[outputs>=0.9] = 1 # cannot change into 1 first\n            #outputs[outputs<0.1] = 0 # cannot change into 0 first\n\n            ret_output += outputs.tolist() #cannot use int() as we need float\n\n    return ret_output #pesudo_x, pesudo_y\n'

# Testing

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1
            outputs[outputs<0.5] = 0
            #print(outputs)
            ret_output += outputs.int().tolist()

    return ret_output

# Parameter setting + Train

In [ ]:
# main.py
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
from sklearn.model_selection import train_test_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set data path
train_with_label = os.path.join(path_prefix, '/content/drive/MyDrive/Dissertation/Train_label.txt')
#train_no_label = os.path.join(path_prefix, '/content/drive/MyDrive/Dissertation/Train_nolabel.txt')
testing_data = os.path.join(path_prefix, '/content/drive/MyDrive/Dissertation/Test.txt')
w2v_path = os.path.join(path_prefix, '/w2v_all.model')


sen_len = 250
fix_embedding = True # fix embedding during training
batch_size = 96
#batch_size = 32
#batch_size = 64
#epoch = 2
epoch = 30
lr = 0.00005 # increase learning rate
model_dir = path_prefix

print("loading data ...")
train_x, y = load_training_data(train_with_label)
#train_x_no_label = load_training_data(train_no_label)

# Preprocessing
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)


model = GRU_Net(embedding, embedding_dim=250, hidden_dim=125, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device)

#X_train, X_val, y_train, y_val = train_x[:130000], train_x[130000:], y[:130000], y[130000:]

X_train, X_val, y_train, y_val = train_test_split(train_x, y, test_size = 0.1, random_state = 1, stratify=y)

train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# transfor data into batch of tensors
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                             batch_size = batch_size,
                             shuffle = True,
                             num_workers = 8)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                           batch_size = batch_size,
                           shuffle = False,
                           num_workers = 8)

#train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
#                            batch_size = batch_size,
#                            shuffle = True,
#                            num_workers = 0)

#val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
#                            batch_size = batch_size,
#                           shuffle = False,
#                           num_workers = 0)

# Begin Training
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)


loading data ...
Get embedding ...
loading word to vec model ...
get words #5765
total words: 5767

start training, parameter total:1583251, trainable:141501

[ Epoch1: 11/11 ] loss:0.633 acc:69.792 
Train | Loss:0.65301 Acc: 67.614
Valid | Loss:0.61267 Acc: 46.354 
saving model with acc 46.354
-----------------------------------------------
[ Epoch2: 11/11 ] loss:0.517 acc:82.292 
Train | Loss:0.60797 Acc: 74.148
Valid | Loss:0.58743 Acc: 46.354 
-----------------------------------------------
[ Epoch3: 11/11 ] loss:0.612 acc:73.958 
Train | Loss:0.60183 Acc: 75.095
Valid | Loss:0.57528 Acc: 46.354 
-----------------------------------------------
[ Epoch4: 11/11 ] loss:0.527 acc:80.208 
Train | Loss:0.59468 Acc: 74.811
Valid | Loss:0.56857 Acc: 46.354 
-----------------------------------------------
[ Epoch5: 11/11 ] loss:0.573 acc:72.917 
Train | Loss:0.58835 Acc: 74.905
Valid | Loss:0.56253 Acc: 46.354 
-----------------------------------------------
[ Epoch6: 11/11 ] loss:0.572 acc

In [ ]:
'''
### Get pesudo label
train_x_no_label_word = load_training_data(train_no_label)
train_x_no_label = preprocess.sentence_word2idx()
preprocess = Preprocess(train_x_no_label, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x_no_label_dataset = TwitterDataset(X=train_x_no_label, y=None)

train_no_loader = torch.utils.data.DataLoader(dataset = train_x_no_label_dataset,
                            batch_size = batch_size,
                            shuffle = False,
                            num_workers = 0)

model = torch.load(os.path.join(model_dir, 'ckpt.model'))
output = got_label(batch_size, train_no_loader, model, device)
'''

"\n### Get pesudo label\ntrain_x_no_label_word = load_training_data(train_no_label)\ntrain_x_no_label = preprocess.sentence_word2idx()\npreprocess = Preprocess(train_x_no_label, sen_len, w2v_path=w2v_path)\nembedding = preprocess.make_embedding(load=True)\ntrain_x_no_label_dataset = TwitterDataset(X=train_x_no_label, y=None)\n\ntrain_no_loader = torch.utils.data.DataLoader(dataset = train_x_no_label_dataset,\n                            batch_size = batch_size,\n                            shuffle = False,\n                            num_workers = 0)\n\nmodel = torch.load(os.path.join(model_dir, 'ckpt.model'))\noutput = got_label(batch_size, train_no_loader, model, device)\n"

In [ ]:
'''
### Pesudo labelled sample selection
#import Tensor

pesudo_label_index = []

#pesudo_x = preprocess.sentence_word2idx()
#preprocess = Preprocess(pesudo_x, sen_len, w2v_path=w2v_path)
#embedding = preprocess.make_embedding(load=True)

for i in range(len(output)):
  if output[i] >= 0.9 or output[i] <= 0.1:
  #if output[i] >= 0.8:
    pesudo_label_index.append(i)

pesudo_x = [train_x_no_label[i] for i in pesudo_label_index]
pesudo_y = [output[i] for i in pesudo_label_index]

for i in range(len(pesudo_y)):
  if pesudo_y[i] <= 0.3:
    pesudo_y[i] = 0.0
  else:
    pesudo_y[i] = 1.0

#print(pesudo_x)

#pesudo_x = TwitterDataset(pesudo_x,  y=None)
#pesudo_x = torch.Tensor(pesudo_x)
'''

'\n### Pesudo labelled sample selection\n#import Tensor\n\npesudo_label_index = []\n\n#pesudo_x = preprocess.sentence_word2idx()\n#preprocess = Preprocess(pesudo_x, sen_len, w2v_path=w2v_path)\n#embedding = preprocess.make_embedding(load=True)\n\nfor i in range(len(output)):\n  if output[i] >= 0.9 or output[i] <= 0.1:\n  #if output[i] >= 0.8:\n    pesudo_label_index.append(i)\n\npesudo_x = [train_x_no_label[i] for i in pesudo_label_index]\npesudo_y = [output[i] for i in pesudo_label_index]\n\nfor i in range(len(pesudo_y)):\n  if pesudo_y[i] <= 0.3:\n    pesudo_y[i] = 0.0\n  else:\n    pesudo_y[i] = 1.0\n\n#print(pesudo_x)\n\n#pesudo_x = TwitterDataset(pesudo_x,  y=None)\n#pesudo_x = torch.Tensor(pesudo_x)\n'

In [ ]:
'''
print("loading data ...")
train_x, y = load_training_data(train_with_label)
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

model = GRU_Net(embedding, embedding_dim=250, hidden_dim=125, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device)

X_train, X_val, y_train, y_val = train_test_split(train_x, y, test_size = 0.05, random_state = 10, stratify=y)

# Combine labelled samples with pesudo labelled samples

pesudo_x = torch.tensor([item.cpu().detach().numpy() for item in pesudo_x]).cuda() #turn list to tensor
pesudo_x = pesudo_x.to(device, dtype=torch.long) # set device "cuda"
X_train = X_train.to(device, dtype=torch.long) # turn list to tensor
retrain_x = torch.cat((X_train,pesudo_x),0)  # combine with label and without label dataset into one
pesudo_y = preprocess.labels_to_tensor(pesudo_y) #turn list to tensor
pesudo_y = torch.Tensor(pesudo_y) # turn list to tensor
pesudo_y = pesudo_y.to(device, dtype=torch.float) # set device "cuda"
y_train = y_train.to(device, dtype=torch.float) # set device "cuda"
retrain_y = torch.cat((y_train,pesudo_y),0)  # combine with label and without label dataset into one
'''

'\nprint("loading data ...")\ntrain_x, y = load_training_data(train_with_label)\npreprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)\nembedding = preprocess.make_embedding(load=True)\ntrain_x = preprocess.sentence_word2idx()\ny = preprocess.labels_to_tensor(y)\n\nmodel = GRU_Net(embedding, embedding_dim=250, hidden_dim=125, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)\nmodel = model.to(device)\n\nX_train, X_val, y_train, y_val = train_test_split(train_x, y, test_size = 0.05, random_state = 10, stratify=y)\n\n# Combine labelled samples with pesudo labelled samples\n\npesudo_x = torch.tensor([item.cpu().detach().numpy() for item in pesudo_x]).cuda() #turn list to tensor\npesudo_x = pesudo_x.to(device, dtype=torch.long) # set device "cuda"\nX_train = X_train.to(device, dtype=torch.long) # turn list to tensor\nretrain_x = torch.cat((X_train,pesudo_x),0)  # combine with label and without label dataset into one\npesudo_y = preprocess.labels_to_tensor(pesudo_y) #turn list 

In [ ]:
#print(pesudo_y)

In [ ]:
'''
### Retrain model with pesudo labelled samples
#batch_size = 192
batch_size = 96
#batch_size = 32
#epoch = 2
#epoch = 5
epoch = 50
#epoch = 7
lr = 0.00005 # use a smaller learning rate

retrain_dataset = TwitterDataset(X=retrain_x, y=retrain_y)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# transfor data into batch of tensors
retrain_loader = torch.utils.data.DataLoader(dataset = retrain_dataset,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 0)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                      batch_size = batch_size,
                      shuffle = False,
                      num_workers = 0)

training(batch_size, epoch, lr, model_dir, retrain_loader, val_loader, model, device)
'''

'\n### Retrain model with pesudo labelled samples\n#batch_size = 192\nbatch_size = 96\n#batch_size = 32\n#epoch = 2\n#epoch = 5\nepoch = 50\n#epoch = 7\nlr = 0.00005 # use a smaller learning rate\n\nretrain_dataset = TwitterDataset(X=retrain_x, y=retrain_y)\nval_dataset = TwitterDataset(X=X_val, y=y_val)\n\n# transfor data into batch of tensors\nretrain_loader = torch.utils.data.DataLoader(dataset = retrain_dataset,\n                          batch_size = batch_size,\n                          shuffle = True,\n                          num_workers = 0)\n\nval_loader = torch.utils.data.DataLoader(dataset = val_dataset,\n                      batch_size = batch_size,\n                      shuffle = False,\n                      num_workers = 0)\n\ntraining(batch_size, epoch, lr, model_dir, retrain_loader, val_loader, model, device)\n'

# Predict and save to csv file

In [ ]:
print("loading testing data ...")
test_x = load_testing_data(testing_data)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                            batch_size = batch_size,
                            shuffle = False,
                            num_workers = 8)

#test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
#                           batch_size = batch_size,
#                           shuffle = False,
#                           num_workers = 0)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, '/content/ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# save as csv
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"labels":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'Dissertation_predict.csv'), index=False)
tmp.to_csv('/content/drive/My Drive/Dissertation/Dissertation_predict_comparsion_gru_supervised_75%.csv', index=False)

#from google.colab import drive
#drive.flush_and_unmount()

print("Finish Predicting")

loading testing data ...
Get embedding ...
loading word to vec model ...
get words #5765
total words: 5767
sentence count #671
load model ...
save csv ...
Finish Predicting


In [ ]:
print(tmp.loc[tmp['labels'] == 1])
print(len(tmp.loc[tmp['labels'] == 1]))

      id  labels
0      0       1
1      1       1
2      2       1
3      3       1
4      4       1
..   ...     ...
666  666       1
667  667       1
668  668       1
669  669       1
670  670       1

[671 rows x 2 columns]
671


In [ ]:

test_verification = pd.read_excel('/content/drive/MyDrive/Dissertation/test_verification.xlsx')
y_true = test_verification['labels']

from sklearn.metrics import f1_score

print(f1_score(y_true, tmp['labels'], average='binary'))

print(len(y_true))
print(len(tmp['labels']))


0.16666666666666669
671
671


In [ ]:
#drive.flush_and_unmount()

from sklearn.metrics import classification_report

print(classification_report(y_true, tmp['labels']))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       610
           1       0.09      1.00      0.17        61

    accuracy                           0.09       671
   macro avg       0.05      0.50      0.08       671
weighted avg       0.01      0.09      0.02       671

